In [2]:
import pandas as pd
import numpy as np

#nilearn imports
from nilearn import plotting, image
from nilearn.image import mean_img
from nilearn.plotting import plot_anat, plot_img, plot_stat_map, show, plot_design_matrix
from nilearn.glm import threshold_stats_img
from nilearn.glm.first_level import FirstLevelModel, make_first_level_design_matrix
from nilearn.reporting import get_clusters_table

In [ ]:
#function to get first level weights for the runs
#possible task types include colorWheel or sameDifferent 

def extract_beta_weights(subject_id = None, task_type = 'colorWheel',  n_runs=1):
    #parameters
    tr = 2.0 
    hrf_model = "spm + derivative"
    smoothing_fwhm = 0.6
    drift_model = "cosine"
    run_betas = {
        'colorWheel': {},
        'sameDifferent': {}
    }

    for num_run in range(n_runs):
        #load subject nii files
        run_img = image.load_img(f'/Volumes/kiyonaga/MIPS/fMRI/subjects/MIPS_{subject_id}/func/run0{num_run+1}/srraf**.nii')

        #load the event file for the run
        events = pd.read_table(f'/Volumes/kiyonaga/MIPS/BIDS/sub-{subject_id}/func/sub-{subject_id}_task-{task_type.lower()}_acq-norm_run-{num_run+1}_events.tsv')

        #run the first level model
        fmri_glm = FirstLevelModel(
            t_r=tr,
            hrf_model=hrf_model,
            smoothing_fwhm=smoothing_fwhm,
            drift_model=drift_model,
            minimize_memory=False   
        )

        fmri_glm = fmri_glm.fit(run_img, events)

        #create design matrix
        design_matrix = fmri_glm.design_matrices_[0]

        #
        n_regressors = design_matrix.shape[1]
        activation = np.zeros(n_regressors)
        activation[0] = 1

        #should we remove confound with high_variance_confounds?

        # t-statistic to z-scale 
        z_map = fmri_glm.compute_contrast(activation, output_type="z_score")
        run_betas[task_type][f'run_{num_run+1}'] = {z_map}
    return run_betas

extract_beta_weights(103, 'sameDifferent')['sameDifferent']['run_1']

/Users/jennifer/anaconda3/lib/python3.11/site-packages/nilearn/glm/first_level/experimental_paradigm.py:129: UserWarning: 'trial_type' column not found in the given events data.
  warnings.warn(
/Users/jennifer/anaconda3/lib/python3.11/site-packages/nilearn/glm/first_level/experimental_paradigm.py:167: UserWarning: The following unexpected columns in events data will be ignored: msecProbeRT, probert, probeACC, stimStart, stimFlip, LastITI, ProbeLoc, error, runStart, scannerStart, SetSize, correctResp, counterbalance, ProbeType, SeqStartTime, CorrectProbeColor, ProbeMatch, chooseSequence, ThisITI, probeStart, centralJitter, trialStart, CurrentTask, subject, response, block, ChosenProbeColor
  warnings.warn(


{<nibabel.nifti1.Nifti1Image at 0x16d3a1050>}

In [20]:


plot_stat_map(
    extract_beta_weights(103, 'sameDifferent')['sameDifferent']['run_1'],
    title="Change Detection Task",
)
show()

/Users/jennifer/anaconda3/lib/python3.11/site-packages/nilearn/glm/first_level/experimental_paradigm.py:129: UserWarning: 'trial_type' column not found in the given events data.
  warnings.warn(
/Users/jennifer/anaconda3/lib/python3.11/site-packages/nilearn/glm/first_level/experimental_paradigm.py:167: UserWarning: The following unexpected columns in events data will be ignored: msecProbeRT, probert, probeACC, stimStart, stimFlip, LastITI, ProbeLoc, error, runStart, scannerStart, SetSize, correctResp, counterbalance, ProbeType, SeqStartTime, CorrectProbeColor, ProbeMatch, chooseSequence, ThisITI, probeStart, centralJitter, trialStart, CurrentTask, subject, response, block, ChosenProbeColor
  warnings.warn(


DimensionError: Input data has incompatible dimensionality: Expected dimension is 3D and you provided a list of 3D images (4D). See https://nilearn.github.io/stable/manipulating_images/input_output.html.